/!\

The code is not clean. I coded a proper version but it got lost after I closed my laptop. Because of a lack of time, the part where I combine different models is not clean at all (I used to have a list of models and a for loop for instance).

/!\

# Imports

In [1]:
import zipfile
import time
import glob
import os
import argparse
from tqdm import tqdm
import PIL.Image as Image
from skimage.io import imsave
import numpy as np
from math import ceil, floor
import copy

import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models import resnet50, resnet152, densenet161, resnext101_32x8d

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data

import random
import cv2
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# Merge train and val

In [0]:
directory = './bird_data'
name_dataset = './bird_data3'

all_images = glob.glob(directory+'/*/*/*')

for k in tqdm(range(len(all_images))):
  
  img = Image.open(all_images[k])


  new_name = all_images[k].replace(directory, name_dataset)
  new_name = new_name.replace('val_images', 'train_images')

  os.makedirs(os.path.dirname(new_name), exist_ok=True)

  img.save(new_name)

100%|██████████| 1702/1702 [10:54<00:00,  1.59it/s]


# Data

In [0]:
data_dir = './bird_data3'

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=Image.BILINEAR),
    transforms.ToTensor(),  # to range [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(data_dir + '/train_images',
                              transform=data_transforms)

batch_size = 64

nfolds = 5
nsamples = len(dataset)
split = nsamples // nfolds
folds = data.random_split(dataset, [split]*(nfolds-1) + [nsamples - split*(nfolds-1)])

# Model

In [0]:
nclasses = 20

class Resnet(nn.Module):
  def __init__(self):
    super(Resnet, self).__init__()
    resnet = resnet50(pretrained=True)
    modules = list(resnet.children())[:-1]
    self.resnet = nn.Sequential(*modules)
    
    self.linear = nn.Linear(resnet.fc.in_features, nclasses)
    self.dropout1 = nn.Dropout(p=0.9)
    
  def forward(self, x):
    x = self.resnet(x)
    x = x.view(x.size(0), -1)
    x = self.dropout1(x)
    x = F.relu(self.linear(x))

    return x


class Net(nn.Module):
  def __init__(self, backbone_net):
    super(Net, self).__init__()

    self.backbone_name = backbone_net
    if backbone_net == 'resnet50':
      backbone = resnet50(pretrained=True)
    elif backbone_net == 'resnet152':
      backbone = resnet152(pretrained=True)
    elif backbone_net == 'densenet161':
      backbone = densenet161(pretrained=True)
    elif backbone_net == 'resnext101_32x8d':
      backbone = resnext101_32x8d(pretrained=True)

    modules = list(backbone.children())[:-1]
    self.backbone = nn.Sequential(*modules)
    
    self.bn1 = nn.BatchNorm1d(1024)
    if backbone_net == 'resnet50' or backbone_net == 'resnet152' or backbone_net == 'resnext101_32x8d':
      self.linear1 = nn.Linear(backbone.fc.in_features, 1024)
    else:
      self.linear1 = nn.Linear(backbone.classifier.in_features, 1024)

    self.linear2 = nn.Linear(1024, nclasses)
    self.dropout1 = nn.Dropout(p=0.9)
    self.dropout2 = nn.Dropout(p=0.4)

    self.classifier = nn.Sequential(self.dropout1, self.linear1, self.bn1, self.dropout2, self.linear2)
    
  def forward(self, x):
    x = self.backbone(x)
    x = x.view(x.size(0), -1)

    x = self.dropout1(x)
    x = F.relu(self.linear1(x))
    x = self.bn1(x)
    x = self.dropout2(x)
    x = F.relu(self.linear2(x))

    return x

In [0]:
def train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs=50, log_interval=10):
  since = time.time()
  val_acc_history = []

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(nepochs):
    for phase in ['train', 'val']:

      if phase == 'train':
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
          if use_cuda:
            data, target = data.cuda(), target.cuda()
          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()
          if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.data.item()))
            
      
      else:
        model.eval()
        validation_loss = 0
        correct = 0
        for data, target in val_loader:
          if use_cuda:
            data, target = data.cuda(), target.cuda()
          output = model(data)
          # sum up batch loss
          validation_loss += criterion(output, target).data.item()
          # get the index of the max log-probability
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        validation_loss /= len(val_loader.dataset)
        validation_accuracy = 100. * correct / len(val_loader.dataset)

        print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
              validation_loss, correct, len(val_loader.dataset),
              validation_accuracy))
        
        if validation_accuracy > best_acc:
          best_acc = validation_accuracy
          best_model_wts = copy.deepcopy(model.state_dict())

    torch.save(model.state_dict(), model_file)
    print('Saved model to ' + model_file + '.\n')

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # load best model weights
  model.load_state_dict(best_model_wts)
  torch.save(model.state_dict(), model_file)
  print('Saved model to ' + model_file + '.\n')
  
  return model, val_acc_history

In [0]:
data_dir = './bird_data3'

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=Image.BILINEAR),
    transforms.ToTensor(),  # to range [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(data_dir + '/train_images',
                              transform=data_transforms)

batch_size = 16

nfolds = 4
nsamples = len(dataset)
split = nsamples // nfolds
folds = data.random_split(dataset, [split]*(nfolds-1) + [nsamples - split*(nfolds-1)])


use_cuda = torch.cuda.is_available()

### INITIALIZATION ###
if use_cuda:
  # model1 = Net('resnet50').cuda()
  # model2 = Net('resnet152').cuda()
  model3 = Net('densenet161').cuda()
  model4 = Net('resnext101_32x8d').cuda()
else:
  model1 = Net('resnet50')
  model2 = Net('resnet152')
  model3 = Net('densenet161')
  model4 = Net('resnext101_32x8d')

# parameters
nepochs = 50
log_interval = 10

# optimization
criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:02<00:00, 50.7MB/s]
Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/checkpoints/resnext101_32x8d-8ba56ff5.pth
100%|██████████| 340M/340M [00:21<00:00, 16.9MB/s]


In [0]:
### TRAINING AND VALIDATION SETS ###
k = 0

train_dataset = copy.deepcopy(folds)
del train_dataset[k]
train_dataset = data.ConcatDataset(train_dataset)
val_dataset = copy.deepcopy(folds[k])

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=False, 
                                            num_workers=1
                                            )

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          num_workers=1
                                          )

if k == 0:
  model = model1
elif k == 1:
  model = model2
elif k == 2:
  model = model3
elif k == 3:
  model = model4

optimizer = optim.Adam([
                      {'params': model.backbone.parameters(), 'lr': 1e-5},
                      {'params': model.classifier.parameters(), 'lr': 1e-4}
                    ])
model_file = 'experiment/bagging/{}.pth'.format(model.backbone_name)

if k == 0:
  model1, val_acc_hist1 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 1:
  model2, val_acc_hist2 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 2:
  model3, val_acc_hist3 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 3:
  model4, val_acc_hist4 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)

Train Epoch: 0 [0/889 (0%)]	Loss: 3.110954
Train Epoch: 0 [160/889 (18%)]	Loss: 3.119203
Train Epoch: 0 [320/889 (36%)]	Loss: 3.159241
Train Epoch: 0 [480/889 (54%)]	Loss: 3.209217
Train Epoch: 0 [640/889 (71%)]	Loss: 3.002697
Train Epoch: 0 [800/889 (89%)]	Loss: 3.205491

Validation set: Average loss: 0.1901, Accuracy: 17/296 (6%)
Saved model to experiment/bagging/resnet50.pth.

Train Epoch: 1 [0/889 (0%)]	Loss: 3.114793
Train Epoch: 1 [160/889 (18%)]	Loss: 2.988680
Train Epoch: 1 [320/889 (36%)]	Loss: 3.047403
Train Epoch: 1 [480/889 (54%)]	Loss: 2.979372
Train Epoch: 1 [640/889 (71%)]	Loss: 2.942901
Train Epoch: 1 [800/889 (89%)]	Loss: 2.889091

Validation set: Average loss: 0.1836, Accuracy: 48/296 (16%)
Saved model to experiment/bagging/resnet50.pth.

Train Epoch: 2 [0/889 (0%)]	Loss: 2.946181
Train Epoch: 2 [160/889 (18%)]	Loss: 2.751307
Train Epoch: 2 [320/889 (36%)]	Loss: 3.149491
Train Epoch: 2 [480/889 (54%)]	Loss: 2.640760
Train Epoch: 2 [640/889 (71%)]	Loss: 2.947680
Train 

In [0]:
### TRAINING AND VALIDATION SETS ###
k = 1

train_dataset = copy.deepcopy(folds)
del train_dataset[k]
train_dataset = data.ConcatDataset(train_dataset)
val_dataset = copy.deepcopy(folds[k])

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=False, 
                                            num_workers=1
                                            )

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          num_workers=1
                                          )

if k == 0:
  model = model1
elif k == 1:
  model = model2
elif k == 2:
  model = model3
elif k == 3:
  model = model4

optimizer = optim.Adam([
                      {'params': model.backbone.parameters(), 'lr': 1e-5},
                      {'params': model.classifier.parameters(), 'lr': 1e-4}
                    ])
model_file = 'experiment/bagging/{}.pth'.format(model.backbone_name)

if k == 0:
  model1, val_acc_hist1 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 1:
  model2, val_acc_hist2 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 2:
  model3, val_acc_hist3 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 3:
  model4, val_acc_hist4 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)

Train Epoch: 0 [0/889 (0%)]	Loss: 3.067159
Train Epoch: 0 [160/889 (18%)]	Loss: 2.897420
Train Epoch: 0 [320/889 (36%)]	Loss: 2.964435
Train Epoch: 0 [480/889 (54%)]	Loss: 3.008861
Train Epoch: 0 [640/889 (71%)]	Loss: 2.895734
Train Epoch: 0 [800/889 (89%)]	Loss: 3.397176

Validation set: Average loss: 0.1877, Accuracy: 35/296 (12%)
Saved model to experiment/bagging/resnet152.pth.

Train Epoch: 1 [0/889 (0%)]	Loss: 3.143836
Train Epoch: 1 [160/889 (18%)]	Loss: 3.029862
Train Epoch: 1 [320/889 (36%)]	Loss: 3.167501
Train Epoch: 1 [480/889 (54%)]	Loss: 2.742148
Train Epoch: 1 [640/889 (71%)]	Loss: 3.083973
Train Epoch: 1 [800/889 (89%)]	Loss: 2.888724

Validation set: Average loss: 0.1783, Accuracy: 57/296 (19%)
Saved model to experiment/bagging/resnet152.pth.

Train Epoch: 2 [0/889 (0%)]	Loss: 2.815286
Train Epoch: 2 [160/889 (18%)]	Loss: 3.084705
Train Epoch: 2 [320/889 (36%)]	Loss: 2.908127
Train Epoch: 2 [480/889 (54%)]	Loss: 2.816869
Train Epoch: 2 [640/889 (71%)]	Loss: 3.004386
Tra

In [0]:
### TRAINING AND VALIDATION SETS ###
from torch.utils import data
k = 2

train_dataset = copy.deepcopy(folds)
del train_dataset[k]
train_dataset = data.ConcatDataset(train_dataset)
val_dataset = copy.deepcopy(folds[k])

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=False, 
                                            num_workers=1
                                            )

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          num_workers=1
                                          )

if k == 0:
  model = model1
elif k == 1:
  model = model2
elif k == 2:
  model = model3
elif k == 3:
  model = model4

optimizer = optim.Adam([
                      {'params': model.backbone.parameters(), 'lr': 1e-5},
                      {'params': model.classifier.parameters(), 'lr': 1e-4}
                    ])
model_file = 'experiment/bagging/{}.pth'.format(model.backbone_name)

if k == 0:
  model1, val_acc_hist1 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 1:
  model2, val_acc_hist2 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 2:
  model3, val_acc_hist3 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 3:
  model4, val_acc_hist4 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)

In [0]:
### TRAINING AND VALIDATION SETS ###
k = 3

train_dataset = copy.deepcopy(folds)
del train_dataset[k]
train_dataset = data.ConcatDataset(train_dataset)
val_dataset = copy.deepcopy(folds[k])

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=False, 
                                            num_workers=1
                                            )

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          num_workers=1
                                          )

if k == 0:
  model = model1
elif k == 1:
  model = model2
elif k == 2:
  model = model3
elif k == 3:
  model = model4

optimizer = optim.Adam([
                      {'params': model.backbone.parameters(), 'lr': 1e-5},
                      {'params': model.classifier.parameters(), 'lr': 1e-4}
                    ])
model_file = 'experiment/bagging/{}.pth'.format(model.backbone_name)

if k == 0:
  model1, val_acc_hist1 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 1:
  model2, val_acc_hist2 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 2:
  model3, val_acc_hist3 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)
elif k == 3:
  model4, val_acc_hist4 = train_model(model, train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)

Train Epoch: 0 [0/888 (0%)]	Loss: 3.135348
Train Epoch: 0 [160/888 (18%)]	Loss: 3.018517
Train Epoch: 0 [320/888 (36%)]	Loss: 3.086227
Train Epoch: 0 [480/888 (54%)]	Loss: 2.958160
Train Epoch: 0 [640/888 (71%)]	Loss: 3.063637
Train Epoch: 0 [800/888 (89%)]	Loss: 3.025895

Validation set: Average loss: 0.1855, Accuracy: 47/297 (16%)
Saved model to experiment/bagging/resnext101_32x8d.pth.

Train Epoch: 1 [0/888 (0%)]	Loss: 2.821014
Train Epoch: 1 [160/888 (18%)]	Loss: 2.926874
Train Epoch: 1 [320/888 (36%)]	Loss: 2.943703
Train Epoch: 1 [480/888 (54%)]	Loss: 3.171844
Train Epoch: 1 [640/888 (71%)]	Loss: 3.244035
Train Epoch: 1 [800/888 (89%)]	Loss: 2.934118

Validation set: Average loss: 0.1710, Accuracy: 115/297 (39%)
Saved model to experiment/bagging/resnext101_32x8d.pth.

Train Epoch: 2 [0/888 (0%)]	Loss: 2.829445
Train Epoch: 2 [160/888 (18%)]	Loss: 2.723266
Train Epoch: 2 [320/888 (36%)]	Loss: 2.906630
Train Epoch: 2 [480/888 (54%)]	Loss: 2.896666
Train Epoch: 2 [640/888 (71%)]	Los

In [0]:
use_cuda = torch.cuda.is_available()

### INITIALIZATION ###
if use_cuda:
  models = [Resnet().cuda()] * nfolds
else:
  models = [Resnet()] * nfolds

# parameters
nepochs = 50
log_interval = 10

# optimization
criterion = nn.CrossEntropyLoss()


for k in range(nfolds):

  ### TRAINING AND VALIDATION SETS ###
  train_dataset = copy.deepcopy(folds)
  del train_dataset[k]
  train_dataset = data.ConcatDataset(train_dataset)
  val_dataset = copy.deepcopy(folds[k])

  train_loader = torch.utils.data.DataLoader(train_dataset, 
                                             batch_size=batch_size, 
                                             shuffle=False, 
                                             num_workers=1
                                             )

  val_loader = torch.utils.data.DataLoader(val_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           num_workers=1
                                           )
  
  optimizer = optim.Adam([
                        {'params': list(models[k].children())[0].parameters(), 'lr': 1e-5},
                        {'params': list(models[k].children())[1].parameters(), 'lr': 1e-4}
                      ])
  model_file = 'experiment/bagging/model_{}.pth'.format(k)
  
  models[k], val_acc_hist = train_model(models[k], train_loader, val_loader, criterion, optimizer, use_cuda, model_file, nepochs, log_interval)

Train Epoch: 0 [0/948 (0%)]	Loss: 3.045092
Train Epoch: 0 [640/948 (67%)]	Loss: 3.083072

Validation set: Average loss: 0.0506, Accuracy: 14/237 (6%)
Saved model to experiment/bagging/model_0.pth.

Train Epoch: 1 [0/948 (0%)]	Loss: 3.115046
Train Epoch: 1 [640/948 (67%)]	Loss: 3.095467

Validation set: Average loss: 0.0506, Accuracy: 13/237 (5%)
Saved model to experiment/bagging/model_0.pth.

Train Epoch: 2 [0/948 (0%)]	Loss: 2.975769
Train Epoch: 2 [640/948 (67%)]	Loss: 3.042772

Validation set: Average loss: 0.0506, Accuracy: 13/237 (5%)
Saved model to experiment/bagging/model_0.pth.

Train Epoch: 3 [0/948 (0%)]	Loss: 2.973353
Train Epoch: 3 [640/948 (67%)]	Loss: 3.037172

Validation set: Average loss: 0.0506, Accuracy: 13/237 (5%)
Saved model to experiment/bagging/model_0.pth.

Train Epoch: 4 [0/948 (0%)]	Loss: 3.050895
Train Epoch: 4 [640/948 (67%)]	Loss: 2.995613

Validation set: Average loss: 0.0506, Accuracy: 13/237 (5%)
Saved model to experiment/bagging/model_0.pth.

Train Epoc

# Evaluate

In [22]:
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

test_dir = data_dir + '/test_images/mistery_category'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


model0 = Resnet().cuda()
model0.load_state_dict(torch.load('experiment/bagging/model_0.pth'))

model1 = Resnet().cuda()
model1.load_state_dict(torch.load('experiment/bagging/model_1.pth'))

model2 = Resnet().cuda()
model2.load_state_dict(torch.load('experiment/bagging/model_2.pth'))

model3 = Resnet().cuda()
model3.load_state_dict(torch.load('experiment/bagging/model_3.pth'))

model4 = Resnet().cuda()
model4.load_state_dict(torch.load('experiment/bagging/model_4.pth'))

model5 = Net('resnet152').cuda()
model5.load_state_dict(torch.load('experiment/bagging/resnet152.pth'))

model6 = Net('resnext101_32x8d').cuda()
model6.load_state_dict(torch.load('experiment/bagging/resnext101_32x8d.pth'))


outfile = 'experiment/bagging/5resnet50_1resnet152_1resnext101.csv'
output_file = open(outfile, "w")
output_file.write("Id,Category\n")
for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = test_transforms(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if use_cuda:
            data = data.cuda()

        model0.eval()
        output0 = model0(data)
        model1.eval()
        output1 = model1(data)
        model2.eval()
        output2 = model2(data)
        model3.eval()
        output3 = model3(data)
        model4.eval()
        output4 = model4(data)
        model5.eval()
        output5 = model5(data)
        model6.eval()
        output6 = model6(data)

        pred0 = output0.data.max(1, keepdim=True)[1]
        pred1 = output1.data.max(1, keepdim=True)[1]
        pred2 = output2.data.max(1, keepdim=True)[1]
        pred3 = output3.data.max(1, keepdim=True)[1]
        pred4 = output4.data.max(1, keepdim=True)[1]
        pred5 = output5.data.max(1, keepdim=True)[1]
        pred6 = output6.data.max(1, keepdim=True)[1]

        vote = np.zeros(nclasses)
        vote[pred0.item()] += 1
        vote[pred1.item()] += 1
        vote[pred2.item()] += 1
        vote[pred3.item()] += 1
        vote[pred4.item()] += 1
        vote[pred5.item()] += 1
        vote[pred6.item()] += 1

        # pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], np.argmax(vote)))

output_file.close()

print("Succesfully wrote " + outfile + ', you can upload this file to the kaggle competition website')

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

  2%|▏         | 5.48M/230M [00:00<00:04, 57.1MB/s]

  4%|▍         | 10.3M/230M [00:00<00:04, 54.9MB/s]

  7%|▋         | 15.5M/230M [00:00<00:04, 54.8MB/s]

  9%|▉         | 21.0M/230M [00:00<00:03, 55.4MB/s]

 12%|█▏        | 27.4M/230M [00:00<00:03, 58.2MB/s]

 14%|█▍        | 32.0M/230M [00:00<00:04, 49.1MB/s]

 16%|█▌        | 36.3M/230M [00:00<00:04, 42.9MB/s]

 18%|█▊        | 42.0M/230M [00:00<00:04, 46.8MB/s]

 20%|██        | 46.5M/230M [00:00<00:04, 46.0MB/s]

 23%|██▎       | 52.1M/230M [00:01<00:03, 49.3MB/s]

 25%|██▌       | 57.7M/230M [00:01<00:03, 51.7MB/s]

 28%|██▊       | 63.9M/230M [00:01<00:03, 55.0MB/s]

 30%|███       | 69.3M/230M [00:01<00:03, 53.2MB/s]

 32%|███▏      | 74.4M/230M [00:01<00:03, 45.1MB/s]

 35%|███▌      | 80.7M/230M [00:01<00:03, 49.7MB/s]

 38%|███▊      | 86.7M/230M 

Succesfully wrote experiment/bagging/5resnet50_1resnet152_1resnext101.csv, you can upload this file to the kaggle competition website
